# Import packages:

In [12]:
from openbiolink.obl2021 import OBL2021Dataset
import torch.nn as nn

# Accessing Data:

In [7]:
obl_dataset = OBL2021Dataset()

Dataset not found, downloading to D:\programming repo-1\university of glasgow\msc project\MSc.Project\obl2021 ...


KGID_HQ_DIR.zip: 45.2MB [00:06, 7.76MB/s]                                                                              


In [9]:
# get the training, validation and test sets

obl_train_dataset = obl_dataset.training
obl_valid_dataset = obl_dataset.validation
obl_test_dataset = obl_dataset.testing

In [11]:
obl_train_dataset[0]

tensor([71575,    22, 61677])

# Embedding Models:

## transE:

In [16]:
class TransE(nn.Module):
    def __init__(self):
        super(TransE, self).__init__()

In [17]:
obj = TransE()